In [1]:
import pandas as pd
import geopandas as gpd
import mytools
import math,simplekml,os

# ----------------------------------空洞制作------------------------------------

# 导入数据和导出目录

In [2]:
out_file = 'e:/'

In [3]:
use = pd.read_excel('d:/linshi/关键空洞详表-地市确认版20200728.xlsx')

In [3]:
data = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/3-拆分后的空洞/覆盖空19481个添加场景区分大中小微.data')

In [5]:
use.shape

(778, 11)

In [6]:
# use = pd.read_clipboard()

In [7]:
data.shape

(19481, 13)

In [8]:
data.head(1)

,city,type,geometry,面积,lon,lat,区域类型,id,制式,distance,标准面积,空洞面积比例,空洞级别
0,十堰,Polygon,"POLYGON ((111.57467 32.59737, 111.57724 32.598...",118802.970802,111.577452,32.595834,农村,kd_1,FDD900,2200.0,1.520531e+07,0.007813,微


In [9]:
data_use = data[['id','geometry']].merge(use,left_on='id',right_on='空洞标识')

In [10]:
data_use.shape

(778, 13)

In [11]:
data_use.columns

Index(['id', 'geometry', '索引', '空洞标识', '空洞面积', '标准面积', '空洞面积/标准基站面积', '  场景',
       '空洞级别', '地市', '区县', '空洞面积/标准基站面积*空洞权重', '实施方案（规划补点方案后续组织测试衔接确认）'],
      dtype='object')

In [12]:
data_use[['lon','lat']] = data_use.apply(lambda x:pd.Series(x['geometry'].centroid.coords[:][0]),axis=1)#

In [4]:
data.columns

Index(['city', 'type', 'geometry', '面积', 'lon', 'lat', '区域类型', 'id', '制式',
       'distance', '标准面积', '空洞面积比例', '空洞级别'],
      dtype='object')

In [14]:
use_columns=['id', 'geometry', '索引', '空洞标识', '空洞面积', '标准面积', '空洞面积/标准基站面积', '  场景',
       '空洞级别', '地市', '区县', '空洞面积/标准基站面积*空洞权重', 'lon',
       'lat']

In [5]:
use_columns =['city', 'type', 'geometry', '面积', 'lon', 'lat', '区域类型', 'id', '制式',
       'distance', '标准面积', '空洞面积比例', '空洞级别']

In [6]:
data_use = data[use_columns]

# 设置变量

In [7]:
#- 变量 - 设置经纬度和参与循环的字段
lon,lat,use_id,city = ['lon','lat','id','city']

In [8]:
#- 颜色 - 按照字符设置颜色
colour_data = '空洞级别'
colour_dic = {'大':'red','中':'yellow','小':'blue','微':'green','other':'white'}

In [9]:
#- 标注 -添加标注列
de_col = use_columns
de_col.remove('geometry')

In [10]:
#- 视野 - 在1400*900的分辨率下  view = 70米对应：125公里（差不多能看到武汉主城区）  700对应1200公里（差不多能看到湖北省）
view = 50000

# 整理数据

In [11]:
data_use.head(1)

,city,type,geometry,面积,lon,lat,区域类型,id,制式,distance,标准面积,空洞面积比例,空洞级别
0,十堰,Polygon,"POLYGON ((111.57467 32.59737, 111.57724 32.598...",118802.970802,111.577452,32.595834,农村,kd_1,FDD900,2200.0,1.520531e+07,0.007813,微


In [12]:
#添加标注列
data_use['description']=''
for inde_1, name_1 in enumerate(de_col):
    data_use['linshi']=de_col[inde_1]+' : '+data_use[de_col[inde_1]].astype('str')+'\n'
    data_use['description'] = data_use['description']+data_use['linshi']
    data_use.drop(columns='linshi')

In [14]:
data_sectors= data_use.copy()
#添加lod的polygon
data_sectors['lod_dis']= view
data_sectors['lon1'] = data_sectors[lon] + data_sectors['lod_dis']*np.sin(45* np.pi/180)*180/( np.pi * 6371229 * np.cos(data_sectors[lat] * np.pi/180))
data_sectors['lat1'] = data_sectors[lat] + data_sectors['lod_dis']*np.cos(45* np.pi/180) / ( np.pi * 6371229 / 180)
data_sectors['lon2'] = data_sectors[lon] + data_sectors['lod_dis']*np.sin(225* np.pi/180)*180/( np.pi * 6371229 * np.cos(data_sectors[lat] * np.pi/180))
data_sectors['lat2'] = data_sectors[lat] + data_sectors['lod_dis']*np.cos(225* np.pi/180) / ( np.pi * 6371229 / 180)
data_sectors['list_data']=[([lon_1, lat_1,lon_2, lat_2]) 
 for lon_1, lat_1,lon_2, lat_2 in zip(data_sectors['lon1'],data_sectors['lat1'],data_sectors['lon2'],data_sectors['lat2'])]

In [15]:
# df = data_sectors.copy()
# #添加颜色列-数字
# df = data_sectors.copy()
# df['colour']=1
# df['colour'] = pd.cut(df['colour'],bins=[-0.1, 0.7, 0.936, 1.1],labels =['red', 'yellow', 'green'])
# df['colour'] = df['colour'].astype('object')
# # df.loc[df[rsrp_count]<100,'colour']='white'

In [16]:
df = data_sectors.copy()
#添加颜色列-文字
df['colour']='white'
for name_col in colour_dic:
    df.loc[df[colour_data]==name_col,'colour'] =colour_dic[name_col]

In [17]:
#最终使用的列
df_use = df.reindex(columns=[city,use_id,colour_data, 'list_data','colour','description','geometry'])   

# 生成图层

In [18]:
def make_kml(data_t,name='红色栅格',cc='ff0000ff',xiankuan=0,namea='grid_id',list_data = 'list_data',description='description',minlodpixels=1):
    print(data_t.shape,cc,xiankuan,namea,list_data,description,minlodpixels)
    style = simplekml.Style()
    style.linestyle.color = simplekml.Color.changealphaint(250, cc)  # 最终线条上色
    style.polystyle.outline = xiankuan
    style.polystyle.color = simplekml.Color.changealphaint(250, cc )  # 最终形状上色
    lod1 = simplekml.Lod(minlodpixels=minlodpixels, maxlodpixels=-1,minfadeextent=None, maxfadeextent=None)
    grid_red = kml.newfolder(name=name)
    for grid,list_data ,description_str,geo in zip(data_t[namea],data_t[list_data],data_t[description],data_t['geometry']):
#         pol_r = grid_red.newpolygon(name=grid,outerboundaryis=list_data[0])#栅格使用
        pol_r = grid_red.newpolygon(name=grid,outerboundaryis=list(geo.exterior.coords),innerboundaryis=[po.coords for po in list(geo.interiors)])
        pol_r.description = description_str
        pol_r.altitudemode = simplekml.AltitudeMode.clamptoground
        lon_dd,lat_dd,lon1_dd,lat1_dd = list_data
        latlonaltbox = simplekml.LatLonAltBox(east =lon_dd ,north=lat_dd ,south=lat1_dd ,west=lon1_dd,
                                              minaltitude=None, maxaltitude=None, altitudemode=None)
        pol_r.region.latlonaltbox = latlonaltbox
        pol_r.region.lod = lod1
        pol_r.style=style

In [19]:
#如果是多个文件就使用这个，一个文件不用
# f = mytools.othern.file_name_paths('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/修正覆盖半径/各个地市覆盖图层/')

In [20]:
# for name_f in f:#如果是多个文件就使用这个，一个文件不用
#     grid_t = pd.read_pickle(name_f)
#     grid_groupby = grid_t.groupby(['所属地区'])

for name_region,data_region in df_use.groupby(colour_data):
    kml = simplekml.Kml()
#         if not os.path.exists('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/修正覆盖半径/各个地市覆盖图层/{}'.format(name_region)):
#             os.makedirs('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/修正覆盖半径/各个地市覆盖图层/{}'.format(name_region))
    file = '{}关键栅格的空洞_{}_{}.kmz'.format(out_file,name_region,data_region.shape[0])
    print('{}开始'.format(file))
    for name_c,data_t in data_region.groupby('colour'):#red:ff0000ff,green:ff008000,yellow:ff00ffff,white:ffffffff,bule:ffff0000
        if 'red' ==name_c:
            make_kml(data_t,name='红色{}个'.format(data_t.shape[0]),cc='ff0000ff',namea=use_id,minlodpixels=50,xiankuan=1)
        elif 'blue' == name_c:
            make_kml(data_t,name='蓝色{}个'.format(data_t.shape[0]),cc='ffff0000',namea=use_id,minlodpixels=50,xiankuan=1)
        elif 'yellow' == name_c:
            make_kml(data_t,name='黄色{}个'.format(data_t.shape[0]),cc='ff00ffff',namea=use_id,minlodpixels=50,xiankuan=1)
        elif 'white' == name_c:
            make_kml(data_t,name='白色{}个'.format(data_t.shape[0]),cc='ffffffff',namea=use_id,minlodpixels=50,xiankuan=1)
        elif 'green'  == name_c:
            make_kml(data_t,name='绿色{}个'.format(data_t.shape[0]),cc='ff008000',namea=use_id,minlodpixels=50,xiankuan=1)
        kml.save(file) if 'kml' in file else kml.savekmz(file, False)
    print('生成图层成功存放在{}'.format(file))

e:/关键栅格的空洞_中_735.kmz开始
(735, 7) ff00ffff 1 id list_data description 50
生成图层成功存放在e:/关键栅格的空洞_中_735.kmz
e:/关键栅格的空洞_大_554.kmz开始
(554, 7) ff0000ff 1 id list_data description 50
生成图层成功存放在e:/关键栅格的空洞_大_554.kmz
e:/关键栅格的空洞_小_762.kmz开始
(762, 7) ffff0000 1 id list_data description 50
生成图层成功存放在e:/关键栅格的空洞_小_762.kmz
e:/关键栅格的空洞_微_17430.kmz开始
(17430, 7) ff008000 1 id list_data description 50
生成图层成功存放在e:/关键栅格的空洞_微_17430.kmz


In [21]:
mytools.sound()
print('自爆')
assert 0 

自爆


AssertionError: 

# ----------------------------------其他制作------------------------------------

# 导入数据和导出目录

In [ ]:
out_file = 'G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/3-小区覆盖区域/'

In [ ]:
data = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/3-小区覆盖区域/农村和非农村合并全网覆盖图层按照地市合并.data')

In [ ]:
data.

In [ ]:
data = data.reset_index()

In [ ]:
data['type'] = data.type

In [ ]:
#多边形的质心
data[['lon','lat']] = data.apply(lambda x:pd.Series(x['geometry'].centroid.coords[:][0]),axis=1)#

In [ ]:
data.head(1)

In [ ]:
data_use = data[['geometry', 'type', 'city', 'lon', 'lat']]

# 设置变量

In [ ]:
#- 变量 - 设置经纬度和参与循环的字段
lon,lat,use_id,city = ['lon','lat','city','city']

In [ ]:
#- 颜色 - 按照字符设置颜色
colour_data = 'city'
colour_dic = {'大':'red','中':'yellow','小':'blue','微':'green','other':'white'}

In [ ]:
#- 标注 -添加标注列
de_col = ['lon','lat','city']

In [ ]:
#- 视野 - 在1400*900的分辨率下  view = 70米对应：125公里（差不多能看到武汉主城区）  700对应1200公里（差不多能看到湖北省）
view = 50000

# 整理数据

In [ ]:
#添加标注列
data_use['description']=''
for inde_1, name_1 in enumerate(de_col):
    data_use['linshi']=de_col[inde_1]+' : '+data_use[de_col[inde_1]].astype('str')+'\n'
    data_use['description'] = data_use['description']+data_use['linshi']
    data_use.drop(columns='linshi')

In [ ]:
data_sectors= data_use.copy()
#添加lod的polygon
data_sectors['lod_dis']= view
data_sectors['lon1'] = data_sectors[lon] + data_sectors['lod_dis']*np.sin(45* np.pi/180)*180/( np.pi * 6371229 * np.cos(data_sectors[lat] * np.pi/180))
data_sectors['lat1'] = data_sectors[lat] + data_sectors['lod_dis']*np.cos(45* np.pi/180) / ( np.pi * 6371229 / 180)
data_sectors['lon2'] = data_sectors[lon] + data_sectors['lod_dis']*np.sin(225* np.pi/180)*180/( np.pi * 6371229 * np.cos(data_sectors[lat] * np.pi/180))
data_sectors['lat2'] = data_sectors[lat] + data_sectors['lod_dis']*np.cos(225* np.pi/180) / ( np.pi * 6371229 / 180)
data_sectors['list_data']=[([lon_1, lat_1,lon_2, lat_2]) 
 for lon_1, lat_1,lon_2, lat_2 in zip(data_sectors['lon1'],data_sectors['lat1'],data_sectors['lon2'],data_sectors['lat2'])]

In [ ]:
# df = data_sectors.copy()
# #添加颜色列-数字
# df = data_sectors.copy()
# df['colour']=1
# df['colour'] = pd.cut(df['colour'],bins=[-0.1, 0.7, 0.936, 1.1],labels =['red', 'yellow', 'green'])
# df['colour'] = df['colour'].astype('object')
# # df.loc[df[rsrp_count]<100,'colour']='white'

In [ ]:
df = data_sectors.copy()
#添加颜色列-文字
df['colour']='white'
# for name_col in colour_dic:
#     df.loc[df[colour_data]==name_col,'colour'] =colour_dic[name_col]

In [ ]:
#最终使用的列
df_use = df.reindex(columns=[city, 'list_data','colour','description','geometry'])   

In [ ]:
df_use = df_use.explode().reset_index(drop=True)

In [ ]:
df_use['type'] = df_use.type

In [ ]:
a = df_use.head(1)['geometry'][0]

In [ ]:
aa = data.reset_index()

In [ ]:
b = aa['geometry'][0]

In [ ]:
b.envelope()

In [ ]:
for po in b.e:
    po.interiors ==True

In [ ]:
a.interiors ==True

# 生成图层

In [ ]:
def make_kml(data_t,name='红色栅格',cc='ff0000ff',xiankuan=0,namea='grid_id',list_data = 'list_data',description='description',minlodpixels=1):
    print(data_t.shape,cc,xiankuan,namea,list_data,description,minlodpixels)
    style = simplekml.Style()
    style.linestyle.color = simplekml.Color.changealphaint(220, cc)  # 最终线条上色
    style.polystyle.outline = xiankuan
    style.polystyle.color = simplekml.Color.changealphaint(125, cc )  # 最终形状上色
    lod1 = simplekml.Lod(minlodpixels=minlodpixels, maxlodpixels=-1,minfadeextent=None, maxfadeextent=None)
    grid_red = kml.newfolder(name=name)
    for grid,list_data ,description_str,geo in zip(data_t[namea],data_t[list_data],data_t[description],data_t['geometry']):
#         pol_r = grid_red.newpolygon(name=grid,outerboundaryis=list_data[0])#栅格使用
        pol_r = grid_red.newmultigeometry(name=grid)
        pol_r.newpolygon(outerboundaryis=list(geo.exterior.coords),innerboundaryis=[po.coords for po in list(geo.interiors)])
        pol_r.description = description_str
        pol_r.altitudemode = simplekml.AltitudeMode.clamptoground
        lon_dd,lat_dd,lon1_dd,lat1_dd = list_data
        latlonaltbox = simplekml.LatLonAltBox(east =lon_dd ,north=lat_dd ,south=lat1_dd ,west=lon1_dd,
                                              minaltitude=None, maxaltitude=None, altitudemode=None)
        pol_r.region.latlonaltbox = latlonaltbox
        pol_r.region.lod = lod1
        pol_r.style=style

In [ ]:
for name_region,data_region in df_use.groupby(city):
    kml = simplekml.Kml()
    file = '{}覆盖区域_{}_{}.kmz'.format(out_file,name_region,data_region.shape[0])
    for name_c,data_t in data_region.groupby('colour'):#red:ff0000ff,green:ff008000,yellow:ff00ffff,white:ffffffff,bule:ffff0000
        if 'red' ==name_c:
            make_kml(data_t,name='红色{}个'.format(data_t.shape[0]),cc='ff0000ff',namea=use_id,minlodpixels=50,xiankuan=1)
        elif 'blue' == name_c:
            make_kml(data_t,name='蓝色{}个'.format(data_t.shape[0]),cc='ffff0000',namea=use_id,minlodpixels=50,xiankuan=1)
        elif 'yellow' == name_c:
            make_kml(data_t,name='黄色{}个'.format(data_t.shape[0]),cc='ff00ffff',namea=use_id,minlodpixels=50,xiankuan=1)
        elif 'white' == name_c:
            make_kml(data_t,name='白色{}个'.format(data_t.shape[0]),cc='ffffffff',namea=use_id,minlodpixels=50,xiankuan=1)
        elif 'green'  == name_c:
            make_kml(data_t,name='绿色{}个'.format(data_t.shape[0]),cc='ff008000',namea=use_id,minlodpixels=50,xiankuan=1)
        kml.save(file) if 'kml' in file else kml.savekmz(file, False)
    print('生成图层成功存放在{}'.format(file))